# 提示语模板

## Mustache

In [ ]:
%pip install chevron

In [1]:
from chevron.renderer import render as mustache_render
from chevron.tokenizer import tokenize as mustache_tokenize

mustache_render(template='Hello, {{ mustache }}! {{ name }}')

'Hello, ! '

## 通过 bindings 提前定义 Template

In [1]:
from illufly.types import Template, Messages, Runnable

t = Template("IDEA", bindings={"task": "hi"})
t.format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\nhi\n'

## 通过 format 提供变量

In [2]:
t.format({"task": "帮我写一首儿歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首儿歌\n'

## 通过绑定 Runnable 提供变量

In [3]:
from illufly.types import Template, Messages, Runnable

class A(Runnable):
    def call(self, prompt: str, **kwargs):
        self._last_input = prompt

a = A()
t = Template(text="你的任务是：{{last_input}}")
a.bind_consumer(t)

a.call("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'

In [22]:
a.provider_dict

{'last_input': '帮我写一首儿歌'}

## 绑定中使用函数

In [23]:
# 映射到深层嵌套的字典变量
Template(
    "IDEA",
    bindings={"task": lambda x: f"请帮我写一首歌"}
).format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

## 绑定时使用映射

In [13]:
class A(Runnable):
    def call(self, prompt: str, **kwargs):
        self._last_input = prompt

a = A()
t = Template(text="你的任务是：{{task}}")
t.bind_provider((a, {"task": "last_input"}))

a.call("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'